In [2]:
import sys,os
import Seqdata    #import seqdata.py file
import numpy as np
import pandas as pd
import time
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Input, Embedding, Masking, Dropout
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
from math import ceil, sqrt
import random
import glob,os
import matplotlib.pyplot as plt
import pickle
#import pdb
import shap
#from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error
# from sklearn.base import BaseEstimator
# from sklearn.model_selection import GridSearchCV
#from sklearn.externals import joblib
#import joblib
import seaborn as sns
import matplotlib.animation as animation

import talos as ta

Using TensorFlow backend.
/home/arash/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/arash/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/arash/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/arash/anaco

In [3]:
# Loading files from drive
folder = '/home/arash/PedVid/PIE/seqdata/'

#train set:

with open(f'{folder}XscaledTrain', 'rb') as f:
     XscaledTrain = pickle.load(f)

with open(f'{folder}YscaledTrain', 'rb') as f:
     YscaledTrain = pickle.load(f)

with open(f'{folder}SscaledTrain', 'rb') as f:
    SscaledTrain = pickle.load(f)

        
with open(f'{folder}userauxTrain', 'rb') as f:
     userauxTrain = pickle.load(f)

#test set:

with open(f'{folder}XscaledTest', 'rb') as f:
     XscaledTest = pickle.load(f)

with open(f'{folder}YscaledTest', 'rb') as f:
     YscaledTest = pickle.load(f)

with open(f'{folder}SscaledTest', 'rb') as f:
    SscaledTest = pickle.load(f)

        
with open(f'{folder}userauxTest', 'rb') as f:
     userauxTest = pickle.load(f)

In [10]:
XscaledTrain.shape
    

(38,)

In [59]:
# Hypter Parameter dict
p = {"Llayers" : [1,2,3],
      "nodes" : [10,50,100],
      "batch_size": [32,128]}

# Define custom metric

def score(y_true, y_pred):
    return -K.mean(K.square(y_pred - y_true))

def VanillaLSTM(X_t,Out_t,X_val,Out_val,params):

    n_features = X_t.shape[2] - 1
    n_steps_in = X_t.shape[1]
    n_steps_out = Out_t.shape[1]
    
    Llayers = params['Llayers']
    nodes = params['nodes']
    batch_size = params['batch_size']

    
    model = Sequential()
    model.add(Masking(mask_value=-0.01, input_shape=(n_steps_in, n_features)))
    
    for i in range(Llayers-1):
        model.add(LSTM(nodes, activation='relu',
                       return_sequences=True))
        
    model.add(LSTM(nodes, activation='relu'))
    model.add(Dense(n_steps_out))


    model.compile(optimizer='adam', loss='mse', metrics = [score])
    
    X_t = X_t[:,:,:n_features]
    X_val = X_val[:,:,:n_features]
    Out_t = Out_t[:,:,0] # 0 is index for X
    Out_val = Out_val[:,:,0]
    
    start = time.time()
    history = model.fit(X_t, Out_t,
                        validation_data=[X_val, Out_val],
                        epochs = 100, verbose = 100,
                        batch_size = batch_size)
    end = time.time()

    #print (f"Finished Fitting Model.\n Time Taken : {end - start} secs")
    
    return history, model

# Evaluate the model and predict
def RMSEerror(inputseq,Out):
    maxX = 1909.8
    minX = 5.065
    n_features = inputseq.shape[2] - 1
    inputseq = inputseq[:,:,:n_features]
    Out = Out[:,:,0] # 0 is index for X

    Out_pred = VLSTM_object.best_model(metric = 'val_score').predict(inputseq)
    
    outinv = np.zeros(Out.shape)
    outpredinv = np.zeros(Out_pred.shape)
    for i in range(len(Out)):
        pos = sum (n>=0 for n in Out[i])     #position of the last actual value on y, not padded
        outinv[i][:pos]= Out[i][:pos] * (maxX-minX) + minX
        outpredinv[i][:pos] = Out_pred[i][:pos]* (maxX-minX) + minX

    RMSEerror = sqrt(mean_squared_error(outpredinv, outinv))    #negative of RMSE

    return RMSEerror


In [60]:
AllVLSTM = pd.DataFrame()
#data prepare
for Dtype in ['T_1_1','T_1_2','T_2_1']:
    
    print (f'Scanning VLSTM for {Dtype}:')
    funclean = Seqdata.PrepareData(dtype = Dtype)
    inputseqTrain, outTrain = funclean.run(XscaledTrain,YscaledTrain,
                                       SscaledTrain,userauxTrain)

    inputseqTest, outTest = funclean.run(XscaledTest,YscaledTest,
                                     SscaledTest,userauxTest)
    
    VLSTM_object = ta.Scan(x = inputseqTrain,
                           y = outTrain, 
                           params = p,
                           model = VanillaLSTM,
                           val_split = 0.2,
                           experiment_name=f'VLSTM{Dtype}')
    
    datadf = VLSTM_object.data
    datadf['Dtype'] = Dtype
    AllVLSTM = AllVLSTM.append(datadf)

  0%|          | 0/18 [00:00<?, ?it/s]

Scanning VLSTM for T_1_1:
Train on 1184 samples, validate on 296 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100


  6%|▌         | 1/18 [00:11<03:10, 11.21s/it]

Train on 1184 samples, validate on 296 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100


 11%|█         | 2/18 [00:24<03:08, 11.77s/it]

...

 89%|████████▉ | 16/18 [06:15<01:03, 31.77s/it]

Train on 880 samples, validate on 220 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
E

 94%|█████████▍| 17/18 [06:37<00:28, 28.80s/it]

Train on 880 samples, validate on 220 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
E

100%|██████████| 18/18 [07:15<00:00, 24.20s/it]


In [62]:
AllVLSTM.to_csv('PIE_HyperParam_VLSTM.csv',index = False)

In [ ]:
p = {"Llayers" : [1,2,3],
     "Dlayers" : [1,2,3],
     "nodes" : [10,50,100],
     "batch_size": [32,128], 
     "drpout" : [0]}

def AuxLSTM(X_t,Out_t,X_val,Out_val,params):
    n_features = X_t.shape[2] - 1
    aux_size = sum(X_t[:,:,n_features][0]!=-0.01)
    n_steps_in = X_t.shape[1]
    n_steps_out = Out_t.shape[1]
    
    Llayers = params['Llayers']
    Dlayers = params['Dlayers']
    drpout = params['drpout']
    nodes = params['nodes']
    batch_size = params['batch_size']

    
    seq_input = Input(shape=(n_steps_in, n_features), dtype='float32', name='seq_input')
    mask = Masking(mask_value=-0.01)(seq_input)
    
    if Llayers==1:
        lstm_out = LSTM(nodes, activation='relu')(mask)
        
    else:
        lstm_out = LSTM(nodes,activation='relu',return_sequences=True)(mask)
        for i in range(Llayers-2):
            lstm_out = LSTM(nodes,activation='relu',return_sequences=True)(lstm_out)
        lstm_out = LSTM(nodes, activation='relu')(lstm_out)

    #output for lstm, corresponds to 0.2 of loss, used to smooth training and regularization:
    sec_output = Dense(n_steps_out, activation='sigmoid' ,name='sec_output')(lstm_out)   

    auxiliary_input = Input(shape=(aux_size,), name='aux_input')
    

    mrgLayer = keras.layers.concatenate([lstm_out, auxiliary_input])

    # We stack a deep densely-connected network on top
    for i in range(Dlayers):
        mrgLayer = Dense(nodes, activation='relu')(mrgLayer)
        if drpout>0:
            mrgLayer = Dropout(drpout)(mrgLayer)

    # And finally we add the main logistic regression layer
    main_output = Dense(n_steps_out, activation='sigmoid', name='main_output')(mrgLayer)

    modelaux = Model(inputs=[seq_input, auxiliary_input], outputs=[main_output, sec_output])


    modelaux.compile(optimizer='adam', loss='mse', metrics = [score],
              loss_weights=[1., 0.2])
    
    
    aux_t = X_t[:,:,n_features][:,-aux_size:]
    aux_val = X_val[:,:,n_features][:,-aux_size:]
    
    X_t = X_t[:,:,:n_features]
    X_val = X_val[:,:,:n_features]
    
    
    Out_t = Out_t[:,:,0] # 0 is index for X
    Out_val = Out_val[:,:,0]
    
    #start = time.time()
    history = modelaux.fit([X_t,aux_t], [Out_t,Out_t], 
                        validation_data = [[X_val, aux_val],[Out_val,Out_val]],
                        epochs = 100,
                        verbose = 10, 
                        batch_size = batch_size)
    
    #end = time.time()

    #print (f"Finished Fitting Model.\n Time Taken : {end - start} secs")
    
    return history, modelaux

# Evaluate the model and predict
def RMSEerror(inputseq,Out):
    maxX = 1909.8
    minX = 5.065
    n_features = inputseq.shape[2] - 1
    aux_size = sum(inputseq[:,:,n_features][0]!=-0.01)
    aux = inputseq[:,:,n_features][:,-aux_size:]
    
    inputseq = inputseq[:,:,:n_features]
    
    Out = Out[:,:,0] # 0 is index for X

    Out_pred = AuxLSTM_object.best_model(
        metric = 'val_main_output_score').predict([inputseq,aux])[0]
    
    outinv = np.zeros(Out.shape)
    outpredinv = np.zeros(Out_pred.shape)
    for i in range(len(Out)):
        pos = sum (n>=0 for n in Out[i])     #position of the last actual value on y, not padded
        outinv[i][:pos]= Out[i][:pos] * (maxXY-minXY) + minXY
        outpredinv[i][:pos] = Out_pred[i][:pos]* (maxXY-minXY) + minXY

    RMSEerror = sqrt(mean_squared_error(outpredinv, outinv))    #negative of RMSE

    return RMSEerror

In [51]:
(1909.8- 5.065)*0.02053 

39.10420955

In [19]:
AllAuxLSTM = pd.DataFrame()
#data prepare
for Dtype in ['T_1_1','T_1_2','T_2_1']:
    
    print (f'Scanning AuxLSTM for {Dtype}:')
    funclean = Seqdata.PrepareData(dtype = Dtype)
    inputseqTrain, outTrain = funclean.run(XscaledTrain,YscaledTrain,
                                       SscaledTrain,userauxTrain)

    inputseqTest, outTest = funclean.run(XscaledTest,YscaledTest,
                                     SscaledTest,userauxTest)
    print(inputseqTrain.shape[0]+inputseqTest.shape[0])
    
#     AuxLSTM_object = ta.Scan(x = inputseqTrain,
#                            y = outTrain, 
#                            params = p,
#                            model = AuxLSTM,
#                            val_split = 0.2,
#                            experiment_name=f'AuxLSTM{Dtype}')
    
#     datadf = AuxLSTM_object.data
#     datadf['Dtype'] = Dtype
#     AllAuxLSTM = AllAuxLSTM.append(datadf)

Scanning AuxLSTM for T_1_1:
2051
Scanning AuxLSTM for T_1_2:
1581
Scanning AuxLSTM for T_2_1:
1581


In [18]:
inputseqTrain[0].shape

(10, 4)

In [68]:
AllAuxLSTM.to_csv('PIE_HyperParam_auxLSTM.csv',index = False)

In [69]:
AllAuxLSTM.head()

,round_epochs,val_loss,val_main_output_loss,val_sec_output_loss,val_main_output_score,val_sec_output_score,loss,main_output_loss,sec_output_loss,main_output_score,sec_output_score,Dlayers,Llayers,batch_size,drpout,nodes,Dtype
0,100,0.001327,0.001310,0.001431,-0.001310,-0.001431,0.001635,0.001321,0.001573,-0.001321,-0.001573,1,1,32,0,10,T_1_1
1,100,0.000676,0.000591,0.000847,-0.000591,-0.000847,0.000807,0.000625,0.000909,-0.000625,-0.000909,1,1,32,0,50,T_1_1
2,100,0.000505,0.000444,0.000620,-0.000444,-0.000620,0.000597,0.000442,0.000771,-0.000442,-0.000771,1,1,32,0,100,T_1_1
3,100,0.002928,0.002483,0.002749,-0.002483,-0.002749,0.003020,0.002550,0.002708,-0.002550,-0.002708,1,1,128,0,10,T_1_1
4,100,0.001232,0.000964,0.001314,-0.000964,-0.001314,0.001576,0.001279,0.001731,-0.001279,-0.001731,1,1,128,0,50,T_1_1
